In [1]:
import os
import pandas as pd
import re

**Loading and merging the data**

In [2]:
# Define the path to the directory containing the CSV files
directory_path = os.path.join('..', 'data', 'mozilla_dataset', 'dataset_before_merge')

# List all CSV files in the directory
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]

# Read the CSV files into DataFrames
dataframes = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
mozilla_dataset = pd.concat(dataframes, ignore_index=True)

**Exploring the data**

In [3]:
print(mozilla_dataset.shape)

(89661, 9)


In [6]:
print(mozilla_dataset.columns)

Index(['Bug ID', 'Type', 'Summary', 'Product', 'Component', 'Assignee',
       'Status', 'Resolution', 'Updated'],
      dtype='object')


In [7]:
mozilla_dataset

,Bug ID,Type,Summary,Product,Component,Assignee,Status,Resolution,Updated
0,1124005,defect,CSS cached in style editor,DevTools,Style Editor,nobody,UNCONFIRMED,---,2023-09-14 13:00:12
1,1312250,defect,"Display non default properties before others, ...",DevTools,DOM,nobody,UNCONFIRMED,---,2024-02-22 23:15:00
2,1447966,enhancement,Add option to temporarily disable CSS features,DevTools,Inspector: Rules,nobody,UNCONFIRMED,---,2022-10-11 14:25:10
3,1457607,defect,"Firefox 59+ version developer tool freeze, can...",DevTools,Console,nobody,UNCONFIRMED,---,2022-10-11 14:32:37
4,1503225,defect,ASAN build - UI freezes when switching from Ne...,DevTools,Console,nobody,UNCONFIRMED,---,2022-10-11 15:09:19
...,...,...,...,...,...,...,...,...,...
89656,1810575,task,Investigate mdn_yari.page_v1 schema errors,Data Platform and Tools,General,nobody,REOPENED,---,2024-01-19 11:21:30
89657,1822095,task,Review buildid SQL queries,Data Platform and Tools,Glean Platform,nobody,REOPENED,---,2023-05-05 05:18:28
89658,1844886,defect,Airflow task bqetl_main_summary .client_probe_...,Data Platform and Tools,General,nobody,REOPENED,---,2023-12-15 10:33:38
89659,1879863,defect,structured missing columns in `firefox_desktop...,Data Platform and Tools,General,nobody,REOPENED,---,2024-02-23 13:04:20


In [8]:
print(mozilla_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89661 entries, 0 to 89660
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Bug ID      89661 non-null  int64 
 1   Type        89661 non-null  object
 2   Summary     89661 non-null  object
 3   Product     89661 non-null  object
 4   Component   89661 non-null  object
 5   Assignee    89661 non-null  object
 6   Status      89661 non-null  object
 7   Resolution  89661 non-null  object
 8   Updated     89661 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.2+ MB
None


**Delete duplicate data [if exist]**

In [9]:
mozilla_dataset.duplicated().any()

False

**generates descriptive statistics of the data**

In [10]:
# generates descriptive statistics of the data
# include="O": This parameter is specifying that you want to include columns with object data type 
mozilla_dataset.describe(include = "O")

,Type,Summary,Product,Component,Assignee,Status,Resolution,Updated
count,89661,89661,89661,89661,89661,89661,89661,89661
unique,3,89555,94,824,1003,4,1,77179
top,defect,ISPDB Database Entries,Core,General,nobody,NEW,---,2023-02-09 07:40:14
freq,55676,12,10000,12951,80538,59910,89661,128


In [11]:
# compute the number of unique values for each column in the DataFrame training_data.
mozilla_dataset.nunique()

Bug ID        89661
Type              3
Summary       89555
Product          94
Component       824
Assignee       1003
Status            4
Resolution        1
Updated       77179
dtype: int64

**Show the unique values for type column**

In [12]:
# Print the unique values for the 'Type' column
type_unique_values = mozilla_dataset['Type'].unique()
print(f"Unique values in the 'Type' column: {type_unique_values}")

Unique values in the 'Type' column: ['defect' 'enhancement' 'task']


**Show the unique values for status column**

In [13]:
# Print the unique values for the 'Status' column
status_unique_values = mozilla_dataset['Status'].unique()
print(f"Unique values in the 'Status' column: {status_unique_values}")

Unique values in the 'Status' column: ['UNCONFIRMED' 'NEW' 'ASSIGNED' 'REOPENED']


In [14]:
'''
Bug ID is unique for each column ==> remove it
Resolution is the same for all rows ==> remove it
Type, status, product, component, updated don't affect our task
'''

"\nBug ID is unique for each column ==> remove it\nResolution is the same for all rows ==> remove it\nType, status, product, component, updated don't affect our task\n"

**Drop unnecessary columns**

In [4]:
# Columns to drop
columns_to_drop = ['Bug ID', 'Type', 'Product', 'Component', 'Status', 'Resolution', 'Updated']

# Drop the specified columns
mozilla_dataset = mozilla_dataset.drop(columns=columns_to_drop)

In [5]:
print(mozilla_dataset.columns)

Index(['Summary', 'Assignee'], dtype='object')


**generates descriptive statistics of the data**

In [6]:
# generates descriptive statistics of the data
# include="O": This parameter is specifying that you want to include columns with object data type 
mozilla_dataset.describe(include = "O")

,Summary,Assignee
count,89661,89661
unique,89555,1003
top,ISPDB Database Entries,nobody
freq,12,80538


**Show the number of nulls in each column**

In [21]:
# Print the number of null values in each column in the training data
print(mozilla_dataset.isnull().sum())

Summary     0
Assignee    0
dtype: int64


**Print the minimum number of occurance of Assignee column**

In [22]:
min_occurrences = mozilla_dataset['Assignee'].value_counts().min()
print(f"The minimum number of occurrences in owner column is {min_occurrences}")

The minimum number of occurrences in owner column is 1


**Filter the dataset do the minimum occurance of each Assignee is 5**

In [7]:
# Calculate the occurrences of each value in the owner column
value_counts = mozilla_dataset['Assignee'].value_counts()

# Filter the dataset to include only rows where the value in 'owner' has at least 5 occurrences
mozilla_dataset = mozilla_dataset[mozilla_dataset['Assignee'].isin(value_counts[value_counts >= 5].index)]

In [8]:
min_occurrences = mozilla_dataset['Assignee'].value_counts().min()
print(f"The minimum number of occurrences in owner column after filteration is {min_occurrences}")

The minimum number of occurrences in owner column after filteration is 5


In [9]:
print(mozilla_dataset.shape)

(88483, 2)


In [29]:
# compute the number of unique values for each column in the DataFrame training_data.
mozilla_dataset.nunique()

Summary     88381
Assignee      304
dtype: int64

**Show the values of Assignee column**

In [30]:
# Print the unique values for the 'Status' column
assignee_unique_values = mozilla_dataset['Assignee'].unique()
print(f"Unique values in the 'Assignee' column:")
for assignee_value in assignee_unique_values:
    print (assignee_value)


Unique values in the 'Assignee' column:
nobody
hmanilla
jimb
poirot.alex
gl
jdescottes
jwatt
daisuke
ayeddi
emilio
kaie
nchevobbe
gijskruitbosch+bugs
continuation
csabou
dlee
martin
arai.unmht
mcs
bugzeeeeee
jboek
kkaya
m_kato
zmckenney
dschubert
mtighe
rob
jcristau
timeless
cdenizet
tthibaud
gregp
jackyzy823
nalexander
bugzillamozillaorg_serge_20140323
hikezoe.birchill
bwerth
krosylight
valentin.gosu
gsvelto
haftandilian
general
gwatson
jmuizelaar
lissyx+mozillians
mozilla
sdowne
chutten
dmosedale
emcminn
gsuntop
halemu
jprickett
mathieu
mconley
nsauermann
pdahiya
scunnane
edilee
enordin
imani
jteow
mcheang
mstriemer
nrishel
nsharpley
rhelmer
sgalich
standard8
teshaq
cgeorgiu
enndeakin
mpohle
ssachdev
choller
jhugman
jneuberger
joschmidt
afranchuk
hsohaney
jules
pmcmanis
ajvincent
brosa
padenot
tomica
wdurand
aryx.bugmail
mh+mozilla
neil
dao+bmo
rpierzina
lgreco
mak
mtigley
bugzilla
itiel_yn8
sphink
mstange.moz
jmaher
zeid
afinder
aglavic
gmierz2
bacasandrei
tom
wptsync
gbrown
james
k

**Remove rows where the 'Assignee' column has the value 'nobody'**

In [10]:
# Remove rows where the 'Assignee' column has the value 'nobody'
mozilla_dataset = mozilla_dataset[mozilla_dataset['Assignee'] != 'nobody']

In [11]:
print(mozilla_dataset.shape)

(7945, 2)


**clean the text in the summary column**

In [12]:
def preprocess_summary(summary):
    # Remove special characters
    summary = re.sub(r'[^\w\s]', '', summary)
    # Remove newlines
    summary = summary.replace('\n', ' ')
    # Remove hyperlinks
    summary = re.sub(r'http\S+', '', summary)
    return summary

In [13]:
# Preprocess the 'Summary' column
mozilla_dataset['Summary'] = mozilla_dataset['Summary'].apply(preprocess_summary)

**Filter the dataset by the number of words in Summary column**

In [14]:
def filter_by_word_count(df, min_word_count):
    return df[df['Summary'].str.split().str.len() >= min_word_count]

In [15]:
# Define the minimum word count
min_word_count = 5

# Filter rows where the 'Summary' has at least min_word_count words
mozilla_dataset = filter_by_word_count(mozilla_dataset, min_word_count)

In [16]:
print(mozilla_dataset.shape)

(7291, 2)


**Save cleaned data**

In [17]:
current_dir = os.getcwd() 
relative_path = os.path.join('..', 'data','mozilla_dataset' ,'cleaned_mozilla_dataset.csv')
mozilla_dataset.to_csv(os.path.join(current_dir, relative_path), index=False) # exclude the DataFrame index from being saved to the CSV file.